In [1]:
cd C:\Users\davidmarx\Documents\Projects\Business Development\Go Fish

C:\Users\davidmarx\Documents\Projects\Business Development\Go Fish


In [2]:
# ipython nbconvert presentation.ipynb --to slides --post serve

In [3]:
# ERI reveal.js styling
from IPython.core.display import HTML
import urllib2

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [4]:
#HTML('<style>' + 
#     urllib2.urlopen('https://raw.githubusercontent.com/ElderResearch/reveal-eri-template/master/css/eri.css').read() +
#     '</style>'
#    )

In [5]:
#HTML('<style>' + 
#     urllib2.urlopen('https://raw.githubusercontent.com/ElderResearch/reveal-eri-template/master/css/reveal.min.css').read() +
#     '</style>'
#    )

In [6]:
#HTML('<style>' + 
#     urllib2.urlopen('https://raw.githubusercontent.com/ElderResearch/reveal-eri-template/master/css/xcode.css').read() +
#     '</style>'
#    )

# Analyzing Reddit

* David Marx @DigThatData
* http://dmarx.github.io
* http://ElderResearch.com

## Projects
* Community Structure
* Community Instability
* Community Evolution
* Regional Peak Activity
* Associating Subreddit Traffic Bursts with Specific Comments
* Conversation Monitoring

# Community Structure

## Reddit as a graph
* Subreddits = Nodes
* Connections = edges

* Different kinds of edges
  * Shared users
  * How subreddits get talked about

* Hierarchical structure

### Guns
![Guns](images/subgraphs/guns.png)

### Minecraft
![Minecraft](images/subgraphs/minecraft.png)

### Pokemon
![Pokemon](images/subgraphs/pokemon.png)

### Washington DC
![Washington DC](images/subgraphs/washingtondc.png)

### Underworld/Drugs
![Underworld/Drugs](images/subgraphs/underworld_drugs.png)

In [7]:
#hideme
from IPython.display import display, HTML
HTML("""
<script>$('div.input').hide();</script>
<iframe 
    src="https://dmarx.github.io/SubredditMentionsGraph/network/" 
    marginwidth="0" 
    marginheight="0"
    width="900" 
    height="900"    
    scrolling="no"
    >
</iframe>
""")

# Community Instability

## Reddit as a dynamic graph
* Window over data, separate graph for each timeslice

## Graph Anomaly Detection
* Calculate statistics on individual nodes in each time slice
* Normalize relative to past and future slices
* Identify local anomalies
* Aggregate statistics on a given time slice to identify anomalous time periods

![images/Reddit_Anomalies.png](images/Reddit_Anomalies.png)

# Community Evolution

## Monitor changes in the dynamic graph over time
* User base
* Community structure

![Active Users](images/Reddit_Active_Users.png)

![Active Subreddits](images/Reddit_Active_Subreddits.png)

![Reddit Communities.png](images/Reddit_Communities.png)

![Reddit Modularity.png](images/Reddit_Modularity.png)

# Regional Activity

## Activity Profiling
* Assume most subscribers of regional subreddits live there
* Construct an activity profile for the region from these users

![DC Reddit Activity](images/dc_reddit_activity.png)

# Associating Subreddit Traffic Bursts with Specific Comments

## SubredditMentions.com
* Scrape comments for "subreddit mentions"
* Overlay on visualization of subreddit traffic

* ... Not actively maintained, backend scraper goes down a lot

In [8]:
HTML("""
<script>$('div.input').hide();</script>
<iframe 
    src="http://subredditmentions.com/" 
    marginwidth="0" 
    marginheight="0"
    width="900" 
    height="1000"    
    scrolling="no"
    >
</iframe>
""")

# Conversation Monitoring

## Hilldog Bot
* Monitor comments containing keywords
* Get a "pulse" for community sentiment on a particular topic
* Potential application: rapid response to negative portrayals of brand

In [9]:
from IPython.display import HTML
mini_hilldog_script = """

<h2>Simplified "hilldog-bot" demo</h2>

<a href="javascript:hilldog_toggle()">Toggle Scraper</a><br>

<a href="javascript:clear_output()">Clear Output</a><br>

SCRAPER IS <span id="scraper-is-running" style="display: none;">NOT</span> RUNNING.<br>

Found <span id="relevant-count">0</span> relevant comments.

<hr>

<div id="reddit-content" style="overflow: auto; height: 400px;"></div>

<script>
var processed_comments = [],
    run_hilldog=true; 
    
function hilldog_toggle() {
  run_hilldog = !run_hilldog;
  
  if(run_hilldog){
    $('#scraper-is-running').hide();
  } else {
    $('#scraper-is-running').show();
  }
}
  
function clear_output() {
  $("#reddit-content").text('');
}

var t = setInterval(function() {
  if(!run_hilldog){return null;};
  
  $.ajax({
      url: 'https://www.reddit.com/r/all/comments.json',
      type: 'GET'
  }).then(
    function(data) {
      var comments = data.data.children,
          commentText;
      for (var key in comments) {
	    post = comments[key];
        commentText = post.data.body;
        if (commentText.toLowerCase().match(/hillary|clinton|hilldog/i) && processed_comments.indexOf(post.data.id) === -1) {

          $("#reddit-content").prepend( '<hr>' );
          $("#reddit-content").prepend( '<br>Comment: <a href="http://reddit.com/comments/'+ post.data.link_id.slice(3) + '/_/' + post.data.id + '?context=3">(link)<br></a>' +  post.data.body );
          $("#reddit-content").prepend( '<br>Author: <a href="http://reddit.com/u/' + post.data.author +'">' + post.data.author + '</a>');
          $("#reddit-content").prepend( '<br>Subreddit: <a href="http://reddit.com/r/' + post.data.subreddit +'">' + post.data.subreddit + '</a>');
          $("#reddit-content").prepend( 'Submission: <a href="' +post.data.link_url+ '">' + post.data.link_title + '</a>' );
          
          processed_comments.push(post.data.id);
          $("#relevant-count").text(processed_comments.length);
          
        }
      }                
    },
    function() {
      console.log("ERROR GETTING COMMENTS");
    }
  );
},2000);
</script>
"""
HTML(mini_hilldog_script)

# Additional topics

## Trending content prediction

* Identify content likely to hit front page
* "Piggy back" submission visibility with early comments

* Closely Related task: identifying trends on twitter

* Proof-of-concept: [/r/risingthreads](https://www.reddit.com/r/risingthreads)

## Optimal Submission characteristics
* Maximize visibility of your client's content
* Time of day, day of the week, title length, title wording, etc.

* Proof-of-concept : 
  * There are good/bad ways to submit
  * Randall Olson's ["Data driven guides"](http://www.randalolson.com/2015/01/11/a-data-driven-guide-to-creating-successful-reddit-posts-redux/)

![Optimal Posting Times](http://www.randalolson.com/wp-content/uploads/pct-above-thrsh-100-august-heatmap-900x282.png)

  * Reddit can be gamed
    * [/r/The_Donald](http://reddit.com/r/the_donald)'s systematic takeover of [/r/all](http://reddit.com/r/all) 
      * ["sticky spamming"](https://www.reddit.com/r/TheoryOfReddit/comments/4fh8s9/this_is_how_rthe_donald_gets_so_many_posts_onto/)
    * [/u/plebiean_lifestyle](http://reddit.com/u/plebiean_lifestyle) - User achieved 100k Submission karma in 8 days
    * [/u/Gallowboob](http://reddit.com/u/Gallowboob) - User's content regularly dominates the front page